In [21]:
import numpy as np

# old function
def jd_to_gst(jd: float, nutation: float) -> float:
    """
    Convert Julian Day (JD) to Greenwich Apparent Sidereal Time (GAST).
    This function calculates the GAST based on the JD and nutation.
    Args:
        jd (float): The Julian Day.
        nutation (float): The nutation in degrees.
    Returns:
        float: The GAST in radians.
    """
    # Approximate Delta T (in days)
    delta_t_days = 32.184 / (24 * 60 * 60)
    # Convert JD(UT) to JD(TT)
    jd_tt = jd + delta_t_days
    # Julian centuries since J2000.0
    t = (jd_tt - 2451545.0) / 36525.0
    
    print("t =",t)

    # Greenwich Mean Sidereal Time (GMST) at 0h UT
    theta_gmst = (
        280.46061837
        + 360.98564736629 * (jd - 2451545.0)
        + 0.000387933 * t**2
        - t**3 / 38710000.0
    )
    # Wrap GMST to [0, 360) range
    theta_gmst = theta_gmst % 360
    # Convert nutation from arcseconds to degrees
    # nutation = nutation / 3600
    # Calculate Greenwich Apparent Sidereal Time (GAST)
    theta_gast = theta_gmst + nutation
    # Wrap GAST to [0, 360) range
    theta_gast = theta_gast % 360
    # Convert to radians
    theta_gast = np.deg2rad(theta_gast)
    # Ensure we return a Python float, not a NumPy type
    return float(theta_gast)


In [19]:
# new function
# UT1-UTC values from Bulletin A (in seconds)
UT1_UTC_TABLE = {
    # MJD: UT1-UTC (seconds)
    61056: 0.072653,  # 2026-01-16
    61057: 0.073264,  # 2026-01-17
    61058: 0.073830,  # 2026-01-18
    61059: 0.074209,  # 2026-01-19
    61060: 0.074336,  # 2026-01-20
    61061: 0.074174,  # 2026-01-21
    61062: 0.073730,  # 2026-01-22
    61063: 0.07306,   # 2026-01-23
    61070: 0.07125,   # 2026-01-30
    61100: 0.06271,   # 2026-03-01
    61130: 0.04545,   # 2026-03-31
    61161: 0.03244,   # 2026-05-01
    61192: 0.03262,   # 2026-06-01
    61222: 0.05041,   # 2026-07-01
    61253: 0.07972,   # 2026-08-01
    61284: 0.09421,   # 2026-09-01
    61314: 0.09499,   # 2026-10-01
    61345: 0.08583,   # 2026-11-01
    61375: 0.08163,   # 2026-12-01
    61406: 0.08471,   # 2027-01-01
}

def jd_to_mjd(jd: float) -> float:
    """Convert Julian Day to Modified Julian Day."""
    return jd - 2400000.5

def get_ut1_utc(jd_utc: float) -> float:
    """
    Get UT1-UTC value for a given JD using linear interpolation.
    
    Args:
        jd_utc (float): Julian Day in UTC
        
    Returns:
        float: UT1-UTC in seconds
    """
    mjd = jd_to_mjd(jd_utc)
    
    mjd_values = np.array(sorted(UT1_UTC_TABLE.keys()))
    ut1_utc_values = np.array([UT1_UTC_TABLE[m] for m in mjd_values])
    
    # Linear interpolation
    ut1_utc = np.interp(mjd, mjd_values, ut1_utc_values)
    
    return ut1_utc

def jd_to_gst_modified(jd_utc: float, nutation: float) -> float:
    """
    Convert Julian Day (UTC) to Greenwich Apparent Sidereal Time (GAST).
    
    Args:
        jd_utc (float): The Julian Day in UTC time scale.
        nutation (float): The equation of the equinoxes in degrees.
    
    Returns:
        float: The GAST in radians.
    """
    # Get UT1-UTC from Bulletin A
    ut1_utc_seconds = get_ut1_utc(jd_utc)  # Lookup from bulletin
    
    # Convert UTC to UT1 (for GMST calculation)
    jd_ut1 = jd_utc + ut1_utc_seconds / 86400.0
    
    # Convert UTC to TT (for the polynomial time parameter)
    delta_t_utc_to_tt = 69.184  # TT - UTC (constant)
    jd_tt = jd_utc + delta_t_utc_to_tt / 86400.0
    
    # Julian centuries since J2000.0 (in TT)
    t = (jd_tt - 2451545.0) / 36525.0
    
    print("t_modified =",t)

    # GMST calculation uses UT1
    theta_gmst = (
        280.46061837
        + 360.98564736629 * (jd_ut1 - 2451545.0)  # Use UT1!
        + 0.000387933 * t**2  # Higher order terms use TT
        - t**3 / 38710000.0
    )
    
    # Wrap GMST to [0, 360) range
    theta_gmst = theta_gmst % 360
    
    # Calculate GAST
    theta_gast = theta_gmst + nutation
    
    # Wrap GAST to [0, 360) range
    theta_gast = theta_gast % 360
    
    # Convert to radians
    theta_gast = np.deg2rad(theta_gast)
    
    return float(theta_gast)

In [20]:
print(jd_to_gst(2461067.76,0))
print(jd_to_gst_modified(2461067.76,0))

t = 0.2607189698151933
3.8422369036807167
t_modified = 0.26071898153978595
3.842242152330074
